In [1]:
# Imports
%matplotlib inline
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import io
import requests
import warnings
warnings.filterwarnings('ignore')

# Load data
url="https://ndownloader.figshare.com/files/10991108"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

# Statistical Analysis

## Test for normal distribution

In order to statistically confirm our hypothesis that the triple pattern type has an impact on the runtime, we first need to check whether the results are normally distributed.

### Test: Normal Distribution

We test whether the samples for each triple pattern type in both evironments for all knowledge graphs differ from a normal distribution.
This function tests the null hypothesis that a sample comes from a normal distribution. It is based on D’Agostino and Pearson’s test that combines skew and kurtosis to produce an omnibus test of normality.

Hypothesis:
$H_0:$ The sample comes from a normal distribution.

In [2]:
# Select data for Study 1 only with cold cache
study1 = df[(df['Studyname'] == "pagesize") & (df['Pagesize'] == 100) & (df['category'] != "empty")]
print("Test for normal distribution")
for env in study1['Setup'].unique():
    print("Setup: {0}".format(env))
    for source in study1['Source'].unique(): 
        print("Source: {0}".format(source))
        for category in study1['category'].unique():
            a, p = stats.mstats.normaltest(study1[(study1['category'] == category) 
                                                    & (study1['Source'] == source)
                                                    & (study1['Setup'] == env)]['ms'])
            print("Category: " + str(category) +  "; p-Value: " + str(p))
    print("============================================")

Test for normal distribution
Setup: Controlled (HDT)
Source: Wiktionary
Category: <v,v,r>; p-Value: 1.29851382259e-77
Category: <v,v,v>; p-Value: 1.69788064554e-276
Category: <r,r,r>; p-Value: 0.0
Category: <v,r,r>; p-Value: 2.61588054726e-122
Category: <r,v,v>; p-Value: 5.55807338e-250
Category: <r,r,v>; p-Value: 3.10023177568e-207
Category: <r,v,r>; p-Value: 6.12115283006e-243
Category: <v,r,v>; p-Value: 0.212635036633
Source: DBpedia
Category: <v,v,r>; p-Value: 8.44424501048e-106
Category: <v,v,v>; p-Value: 8.6194508316e-244
Category: <r,r,r>; p-Value: 3.49609770398e-265
Category: <v,r,r>; p-Value: 1.06041061823e-139
Category: <r,v,v>; p-Value: 3.83910616812e-205
Category: <r,r,v>; p-Value: 0.0
Category: <r,v,r>; p-Value: 3.77576943638e-276
Category: <v,r,v>; p-Value: 3.44452814178e-17
Source: GeoNames
Category: <v,v,r>; p-Value: 1.5331837e-47
Category: <v,v,v>; p-Value: 6.37007179235e-245
Category: <r,r,r>; p-Value: 1.83036198374e-212
Category: <v,r,r>; p-Value: 3.8212212849e-42
Ca

### Result
For all triple pattern types, we can reject the null hypothesis at a significance level of $\alpha = 0.05$ for all knowledge graphs in the real-world environment. For the controlled environment, we cannot reject the null hypothesis for the pattern type $\langle v,r,v \rangle$ for DBLP, Wiktionary in the controlled environment with HDT backend, for GeoNames in the controlled environment with Virtuoso backend and for Wiktionary and DBLP in the real-world environment. However, the due to the rather low $\alpha$ values and with 91 / 96 rejected hypothesis, we assume the samples do not come from a normal distribution. Conclusively, we apply a non-parametric test to examine the different pattern types.

## Triple Pattern Type.

To test if the samples for the different pattern types significantly differ, we apply the Kruskal-Wallis test.
The test can be used to study, whether a group of samples originates from the same distrbution.

### Test: Kruskal Wallis Test

We apply the Kruskal Wallis test to test whether there is a significant difference between the group. The response times for a specific Triple Pattern type represent a group.

Hypothesis:
$H_0$: There is no difference between the groups

### Response time

In [3]:
# Split the data into the categories
for env in study1['Setup'].unique():
    print("Setup: {0}".format(env))
    for source in study1['Source'].unique(): 
        print("Knowledge graph: {0}".format(source))
        col = 'ms'
        cats = study1['category'].unique()
        t_df_1 = study1[(study1['category'] == cats[0]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        t_df_2 = study1[(study1['category'] == cats[1]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        t_df_3 = study1[(study1['category'] == cats[2]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        t_df_4 = study1[(study1['category'] == cats[3]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        t_df_5 = study1[(study1['category'] == cats[4]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        t_df_6 = study1[(study1['category'] == cats[5]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        t_df_7 = study1[(study1['category'] == cats[6]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        t_df_8 = study1[(study1['category'] == cats[7]) & (study1['Source'] == source) & (study1['Setup'] == env)]
        stat, p = stats.kruskal(t_df_1[col],t_df_2[col], t_df_3[col], t_df_4[col], t_df_5[col], t_df_6[col], t_df_7[col], t_df_8[col])
        print("Kruskal Wallis Test: p-Value = " + str(p))
    print("============================================")

Setup: Controlled (HDT)
Knowledge graph: Wiktionary
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: DBpedia
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: GeoNames
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: DBLP
Kruskal Wallis Test: p-Value = 0.0
Setup: Real-World (HDT)
Knowledge graph: Wiktionary
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: DBpedia
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: GeoNames
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: DBLP
Kruskal Wallis Test: p-Value = 0.0
Setup: Controlled (Virtuoso)
Knowledge graph: Wiktionary
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: DBpedia
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: GeoNames
Kruskal Wallis Test: p-Value = 0.0
Knowledge graph: DBLP
Kruskal Wallis Test: p-Value = 0.0


### Result 

The Kruskal Wallis test yields a p-Value of 0.0 which means that the Null Hypothesis may be rejected at a significance level of $\alpha = 0.05$ and thus there is a significant difference between the groups. The next step is a post-hoc analysis to find which pair of types differ from each other.

## Answer Cardinality.

Next, we want to study the relation between the answer cardinality and the response time. For this purpose, we conduct a correlation analysis.

In [4]:
study2 = df[(df['Studyname'] == "pagesize") & (df['category'] != "empty")]
# Gather data
rho = []
for env in study2['Setup'].unique():
    for pagesize in study2[study2['Setup'] == env]['Pagesize'].unique():
        for server in list(study2['Source'].unique()):
            a = study2[(study2['Source'] == server) 
                       & (study2['category'] != "<v,v,v>")
                       & (study2['Setup'] == env)
                      & (study2['Pagesize'] == pagesize)]
            one_page = a[a['one_page'] == True] 
            more_pages = a[a['one_page'] == False] 
            rho_1 = stats.pearsonr(one_page['total_items'], one_page['ms'])[0]
            rho_2 = stats.pearsonr(more_pages['total_items'], more_pages['ms'])[0]
            rho_all = stats.pearsonr(a['total_items'], a['ms'])[0]
            rho.append({"KG": server, "Setup" : env, "r_1" : rho_1, "r_2" : rho_2, "r_all" : rho_all, "Pagesize" : pagesize })
# Create Table
cdf = pd.DataFrame(rho)
cdf.sort_values(by=['KG', 'Setup', 'Pagesize'], inplace=True)
#cdf = cdf.transpose()
cdf

,KG,Pagesize,Setup,r_1,r_2,r_all
3,DBLP,100,Controlled (HDT),0.265837,0.099680,0.099575
11,DBLP,500,Controlled (HDT),0.141163,0.261873,0.051090
15,DBLP,1000,Controlled (HDT),0.857879,0.229049,0.268997
7,DBLP,10000,Controlled (HDT),0.978985,0.048992,0.475229
23,DBLP,100,Controlled (Virtuoso),0.078326,0.441284,0.165816
31,DBLP,500,Controlled (Virtuoso),0.842352,0.884927,0.522419
35,DBLP,1000,Controlled (Virtuoso),0.913101,0.895921,0.419663
27,DBLP,10000,Controlled (Virtuoso),0.733699,0.727156,0.451176
19,DBLP,100,Real-World (HDT),0.039026,-0.059120,0.003906
1,DBpedia,100,Controlled (HDT),0.296802,-0.014395,0.035978


## Page Size.

In the following, we calculate the relative difference in throughput, when increasing the page size.

In [5]:
study5 = df[(df['Studyname'] == "pagesize") & (df['Environment'] == "Controlled")]
g = pd.pivot_table(study5, values='throughput', index=['Source', 'Backend'], columns=['Pagesize'], aggfunc=np.mean)
g['100-500'] = ((g[500] - g[100]) / g[500]).apply(lambda x: '%.3f' % x)
g['500-1000'] = ((g[1000] - g[500]) / g[1000]).apply(lambda x: '%.3f' % x)
g['1000-10000'] = ((g[10000] - g[1000]) / g[10000]).apply(lambda x: '%.3f' % x)
g = g.drop([100,500, 1000, 10000], axis=1)
g_t = g.transpose()
g_t

Source        DBLP          DBpedia          GeoNames          Wiktionary  \
Backend        HDT Virtuoso     HDT Virtuoso      HDT Virtuoso        HDT   
Pagesize                                                                    
100-500      0.677    0.329   0.636    0.380    0.751    0.376      0.710   
500-1000     0.174    0.052   0.274    0.031    0.175   -0.047      0.124   
1000-10000  -0.072    0.033  -0.013   -0.119   -0.011    0.115     -0.085   

Source               
Backend    Virtuoso  
Pagesize             
100-500       0.345  
500-1000      0.029  
1000-10000    0.083

Next, we apply a Kruskal-Wallis test to determine the whether the difference in throughput are statistically significant.

In [6]:
# Select the data
for setup in study5['Setup'].unique():
    print("Setup: {0}".format(setup))
    for source in study5['Source'].unique():
        col = "throughput"
        p100 = study5[(study5['Source'] == source) & (study5['Pagesize'] == 100) &
                          (study5['Setup'] == setup)]
        p500 = study5[(study5['Source'] == source) & (study5['Pagesize'] == 500) &
                          (study5['Setup'] == setup)]
        p1000 = study5[(study5['Source'] == source) & (study5['Pagesize'] == 1000) &
                          (study5['Setup'] == setup)]
        p10000 = study5[(study5['Source'] == source) & (study5['Pagesize'] == 10000) &
                          (study5['Setup'] == setup)]
        stat, p = stats.kruskal(p100[col],p500[col], p1000[col], p10000[col])
        print("Knowledge Graph: {0}".format(source))
        print("Kruskal Wallis Test: p-Value = " + str(p))
    print("============================================")

Setup: Controlled (HDT)
Knowledge Graph: Wiktionary
Kruskal Wallis Test: p-Value = 3.14376998994e-45
Knowledge Graph: DBpedia
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: GeoNames
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: DBLP
Kruskal Wallis Test: p-Value = 0.0
Setup: Controlled (Virtuoso)
Knowledge Graph: Wiktionary
Kruskal Wallis Test: p-Value = 0.00303102673078
Knowledge Graph: DBpedia
Kruskal Wallis Test: p-Value = 2.79489895375e-82
Knowledge Graph: GeoNames
Kruskal Wallis Test: p-Value = 1.60609385667e-35
Knowledge Graph: DBLP
Kruskal Wallis Test: p-Value = 6.53905633292e-07


### Result

The Kruskal Wallis test yields a p-Value of 0.0 which means that the Null Hypothesis may be rejected at a significance level of  α=0.05 and thus there is a significant difference between the groups, i.e. the throughput for the different page size differ significantly.

## Paginating.


In [7]:
# Select the data
study4 = df[(df['Studyname'] == "paginating")]

for backend in study4['Backend'].unique():
    a = study4[study4['Backend'] == backend]
    rho = stats.pearsonr(a['page'], a['ms'])[0]
    print("Correlation coefficient for paginating with {0} backend: rho = {1}".format(backend, rho))

Correlation coefficient for paginating with HDT backend: rho = -0.0358997315066
Correlation coefficient for paginating with Virtuoso backend: rho = 0.920401015433


### Result

The correlation coeffiecents indicate that for the VIrtuoso backend there is a strong positive correletion between the page number and the response time. Moreover, the results show, taht this is not true for the HDT backend.

## KG / TPF Instance Relation.

We conduct a Kruskal-Wallis test to check for the statistical significance of our results. This time we compare for each environment and each source whether therer is a difference between the uncached and cached results. 

### Test: Kruskal Wallis Test

In [8]:
# Select the data
study3 = df[(df['Studyname'] == "all_loaded")]

for setup in study3['Setup'].unique():
    print("Setup: {0}".format(setup))
    for source in study3['Source'].unique():
        single_kg = study3[(study3['Source'] == source) & (study3['KGs'] == "Single KG") &
                          (study3['Setup'] == setup)]
        multiple_kg = study3[(study3['Source'] == source) & (study3['KGs'] == "Multiple KGs") &
                          (study3['Setup'] == setup)]
        stat, p = stats.kruskal(single_kg, multiple_kg)
        print("Knowledge Graph: {0}".format(source))
        print("Kruskal Wallis Test: p-Value = " + str(p))
    print("============================================")

Setup: Controlled (HDT)
Knowledge Graph: Wiktionary
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: DBpedia
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: GeoNames
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: DBLP
Kruskal Wallis Test: p-Value = 0.0
Setup: Controlled (Virtuoso)
Knowledge Graph: Wiktionary
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: DBpedia
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: GeoNames
Kruskal Wallis Test: p-Value = 0.0
Knowledge Graph: DBLP
Kruskal Wallis Test: p-Value = 0.0


### Result

We find that the response time is significantly different at a level of $\alpha = 0.05$ when comparing the results for a single KG loaded on the TPF server to the results of multiple KGs loaded on the TPF server.